## 0. Import

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import nbinom, poisson, binom
import pymc as pm
import arviz as az
from Bio import Phylo
from tqdm.notebook import tqdm
from Bio import SeqIO

WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## 1.combine one N with one T

In [2]:
def concat_phyAfa(Ns, Ts, newNameT,outNameT, INDIR, OUTDIR, numUC=3, mutFreq=0.6, numMuts_N=2, numMuts_T=2):
    """
    concat multi ple normal and tumors phy/fasta into one
    add sampleID before cell barcode: ACAAAAGCTGGTCCAGACAGA -> N_ACAAAAGCTGGTCCAGACAGA
    """
    for idx, x in enumerate(Ns):
        print(x)
        ## input N
        N = pd.read_csv(INDIR+x+"_"+str(numUC)+"_"+str(mutFreq)+".phy",sep = " ",names = ["name","bi"],skiprows = 2)
        mutN = pd.read_csv(INDIR+x+"_"+str(numUC)+"_"+str(mutFreq)+"_mutations.txt",sep = "\t",names = ["cellBC","numRead","numMut","Del"],skiprows = 0)
        fastaN = INDIR+x+"_"+str(numUC)+"_"+str(mutFreq)+".fa"
        selectedN=mutN[mutN["numMut"]>=2].cellBC
        N = N[N["name"].isin(selectedN)]
        N["name"] = "N_"+N["name"]
        
        ## input T
        T = pd.read_csv(INDIR+Ts[idx]+"_"+str(numUC)+"_"+str(mutFreq)+".phy",sep = " ",names = ["name","bi"],skiprows = 2)
        mutT = pd.read_csv(INDIR+Ts[idx]+"_"+str(numUC)+"_"+str(mutFreq)+"_mutations.txt",sep = "\t",names = ["cellBC","numRead","numMut","Del"],skiprows = 0)
        fastaT = INDIR+Ts[idx]+"_"+str(numUC)+"_"+str(mutFreq)+".fa"
        selectedT=mutT[mutT["numMut"]>=2].cellBC
        T = T[T["name"].isin(selectedT)]
        T["name"] = newNameT[idx]+"_"+T["name"]

        ## output fasta
        output_fasta= open(OUTDIR+outNameT[idx]+"_N2T2.fasta","w")
        for record in SeqIO.parse(ref, "fasta"):
            record.name=record.description=record.id="ref"
            SeqIO.write(record, output_fasta, "fasta-2line")
        for record in SeqIO.parse(fastaN, "fasta"):
            if record.id in set(selectedN):
                record.name=record.description=record.id = "N_"+record.id
                SeqIO.write(record, output_fasta, "fasta-2line")
        for record in SeqIO.parse(fastaT, "fasta"):
            if record.id in set(selectedT):
                record.name=record.description=record.id = newNameT[idx]+"_"+record.id
                SeqIO.write(record, output_fasta, "fasta-2line")
        output_fasta.close()

        ## output phy
        M = pd.concat([N,T])
        new_row = pd.DataFrame({'name':str(len(M)+1), 'bi': 3004},index = [0])
        ref_row = pd.DataFrame({'name':"ref", 'bi': "".join(["0"]*3004)},index = [1])
        new = pd.concat([new_row,ref_row,M.loc[:]]).reset_index(drop=True)
        new.to_csv(OUTDIR+outNameT[idx]+"_N"+str(numMuts_N)+"T"+str(numMuts_T)+".phy",sep = " ",index=False,header = None)


In [4]:
"""
combine one N with one T
"""
INDIR="/data2/kantian/LineageTracing/SMALT/0.Results0.99/0.5.consensus0.99/"
OUTDIR="/data2/kantian/LineageTracing/SMALT/0.Results/1.N2T2/"
ref="/data2/kantian/LineageTracing/SMALT/reference/3k_HMFonly.fasta"

############################################################################################################
############################################################################################################
Ns=["4_N","50_N"]
Ts=['4_T','50_T']

newNameT=['T','T']
outNameT=['IBD4_T','IBD50_T']

numUC=3                  ## number of ccs reads in one consensus cluster
mutFreq=0.6              ## export the mutation if mutation frequency >= mutFreq 
numMuts_N = 2            ## minimum number of mutations in each CCS reads
numMuts_T = 2

concat_phyAfa(Ns, Ts, newNameT,outNameT, INDIR, OUTDIR, numUC=3, mutFreq=0.6, numMuts_N=2, numMuts_T=2)


17_N
151_N
